<a href="https://colab.research.google.com/github/animish16/Face-Detection-and-Comparing-Two-Faces-using-ResNet-18-and-Cosine-Similarity/blob/main/hw2p2_recitation_ResNet18.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Initializations
___

* Custom Dataset & DataLoader
* Torchvision ImageFolder Dataset
* Residual Block
* CNN model with Residual Block
* Loss Functions (Center Loss and Triplet Loss)

## Imports

In [ ]:
# Initializations
import os
import numpy as np
from PIL import Image
import torch
import torchvision
import torchvision.transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data
from torch.utils.data import Dataset, DataLoader
# import torch.optim as optim
# cuda = torch.cuda.is_available()
# import matplotlib.pyplot as plt
import pytz
import time
from tqdm import tqdm
from datetime import datetime
import csv

## Unzipping and folder restructuring

In [ ]:
# # Data unzipping and re-arrangement
# !unzip 11-785-s20-hw2p2-classification.zip
# !rm 11-785-s20-hw2p2-classification.zip
# !tar -zxvf 11-785-hw2p2-s20.tgz
# !rm 11-785-hw2p2-s20.tgz
# !mv 11-785hw2p2-s20/* ~/hw2p2

# !unzip -q test_classification.zip
# !unzip -q test_verification.zip
# !unzip -q train_data.zip
# !unzip -q validation_classification.zip
# !unzip -q validation_verification.zip

# !rm test_classification.zip
# !rm test_verification.zip
# !rm train_data.zip
# !rm validation_classification.zip
# !rm validation_verification.zip
# !rm -r __MACOSX
# !rm -r 11-785hw2p2-s20

## Print layer details

In [ ]:
# class PrintLayer(nn.Module):
#     def __init__(self):
#         super(PrintLayer, self).__init__()
        
#     def forward(self, x):
#         print(str(x.shape))
#         return x

## Residual Block

Resnet: https://arxiv.org/pdf/1512.03385.pdf

Here is a basic usage of shortcut in Resnet

In [ ]:
class BasicBlock(nn.Module):
    expansion = 1
    def __init__(self, channel_size, channel_size_out, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(channel_size, channel_size_out, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(channel_size_out)
        self.conv2 = nn.Conv2d(channel_size_out, channel_size_out, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(channel_size_out)
        if stride > 1:
            self.shortcut = nn.Sequential(
                nn.Conv2d(channel_size, channel_size_out, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(channel_size_out)
            )
        else:
            self.shortcut = None
    def forward(self, x):
        identity = x
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        if self.shortcut:
            identity = self.shortcut(identity)
        out += identity
        out = F.relu(out)
        return out

## Bottleneck (Not Required)

In [ ]:
# class Bottleneck(nn.Module):
#     expansion = 4

#     def __init__(self, channel_size, channel_size_out, stride=1):
#         # channel_size_out = channel_size
#         super(Bottleneck, self).__init__()
#         self.conv1 = nn.Conv2d(channel_size, channel_size_out, kernel_size=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(channel_size_out)
#         self.conv2 = nn.Conv2d(channel_size_out, channel_size_out, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(channel_size_out)
#         self.conv3 = nn.Conv2d(channel_size_out, self.expansion*channel_size_out, kernel_size=1, bias=False)
#         self.bn3 = nn.BatchNorm2d(self.expansion*channel_size_out)

#         self.shortcut = nn.Sequential()
#         if stride != 1 or channel_size != self.expansion*channel_size_out:
#             self.shortcut = nn.Sequential(
#                 nn.Conv2d(channel_size, self.expansion*channel_size_out, kernel_size=1, stride=stride, bias=False),
#                 nn.BatchNorm2d(self.expansion*channel_size_out)
#             )

#     def forward(self, x):
#         out = F.relu(self.bn1(self.conv1(x)))
#         out = F.relu(self.bn2(self.conv2(out)))
#         out = self.bn3(self.conv3(out))
#         out += self.shortcut(x)
#         out = F.relu(out)
#         return out

## ResNet

In [ ]:
class ResNet(nn.Module):
    def __init__(self, layer = BasicBlock, num_layers = [2, 2, 2, 2], num_classes=2300):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(layer, 64, num_layers[0], stride=1)
        self.layer2 = self._make_layer(layer, 128, num_layers[1], stride=2)
        self.layer3 = self._make_layer(layer, 256, num_layers[2], stride=2)
        self.layer4 = self._make_layer(layer, 512, num_layers[3], stride=2)
        self.linear = nn.Linear(8192, num_classes)
        
        # For creating the embedding to be passed into the Center Loss criterion
        self.linear_closs = nn.Linear(8192, 128, bias=False)
        self.relu_closs = nn.ReLU(inplace=True)

    def _make_layer(self, layer, planes, num_layers, stride):
        strides = [stride] + [1]*(num_layers-1)
        layers = []
        for stride in strides:
            layers.append(layer(self.in_planes, planes, stride))
            self.in_planes = planes * layer.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        output = F.relu(self.bn1(self.conv1(x)))
        output = self.layer1(output)
        output = self.layer2(output)
        output = self.layer3(output)
        output = self.layer4(output)
        output = output.view(output.size(0), -1)
        label_output = self.linear(output)
        
        # Create the feature embedding for the Center Loss
        closs_output = self.linear_closs(output)
#         closs_output = self.relu_closs(closs_output)
        
        return closs_output, label_output


### Training & Testing Model

In [ ]:
def train(model, data_loader, test_loader, task='Classification'):
    model.train()

    for epoch in range(numEpochs):
        print("Train: ", datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d %H:%M"))
        avg_loss = 0.0
        print_batch = len(data_loader) // 20
        batch_display = tqdm(enumerate(data_loader), total=len(data_loader), desc="current loss: 0", leave=True, position=0)
        for batch_num, (feats, labels) in batch_display:
            feats, labels = feats.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(feats)[1]
            # outputs = outputs[1]

            # loss = criterion(outputs, labels.long())
            loss = criterion(outputs, labels.long())
            loss.backward()
            optimizer.step()
            
            avg_loss += loss.item()

            if batch_num % print_batch == (print_batch - 1):
                batch_display.set_description("current loss: %.4f" % (avg_loss / print_batch))
                avg_loss = 0.0
            
            torch.cuda.empty_cache()
            del feats
            del labels
            del loss
        
#         print("Val: ", datetime.now(pytz.timezone('US/Eastern')).strftime("%Y-%m-%d %H:%M"))
        if task == 'Classification':
            val_loss, val_acc = test_classify(model, test_loader)
            train_loss, train_acc = test_classify(model, data_loader)
            print('Train Loss: {:.4f}\tTrain Accuracy: {:.4f}\tVal Loss: {:.4f}\tVal Accuracy: {:.4f}'.
                  format(train_loss, train_acc, val_loss, val_acc))
#         else:
#             test_verify(model, test_loader)
            
        torch.save(model.state_dict(), "model_e" + str(epoch + 1) + "_" + datetime.now(pytz.timezone('US/Eastern')).strftime("%Y%m%d%H%M") + ".pt")


def test_classify(model, test_loader):
    model.eval()
    test_loss = []
    accuracy = 0
    total = 0

    for batch_num, (feats, labels) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)[1]
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        
        # loss = criterion(outputs, labels.long())
        loss = criterion(outputs, labels.long())
        
        accuracy += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
        test_loss.extend([loss.item()]*feats.size()[0])
        del feats
        del labels

    model.train()
    return np.mean(test_loss), accuracy/total


# def test_verify(model, test_loader):
#     raise NotImplemented

## Dataset, DataLoader and Constant Declarations

#### Weights

In [ ]:
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear:
        torch.nn.init.xavier_normal_(m.weight.data)

#### Training transformations

In [ ]:
transform_train = torchvision.transforms.Compose([
#     transforms.RandomCrop(32, padding=4),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor()
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

#### Load datasets

In [ ]:
train_dataset = torchvision.datasets.ImageFolder(root='train_data/medium/', transform=transform_train)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=32)

dev_dataset = torchvision.datasets.ImageFolder(root='validation_classification/medium/', transform=transform_train)
dev_dataloader = torch.utils.data.DataLoader(dev_dataset, batch_size=128, shuffle=True, num_workers=32)

#### Set hyperparameters

In [ ]:
numEpochs = 5
num_feats = 3

learningRate = 5e-2
weightDecay = 5e-5

hidden_sizes = [32, 64]
num_classes = len(train_dataset.classes)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# network = Network(num_feats, hidden_sizes, num_classes)
# network.apply(init_weights)
network = ResNet()
network.apply(init_weights)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
# optimizer = torch.optim.Adam(network.parameters(), lr=learningRate, weight_decay=weightDecay)

#### Load a saved network

In [ ]:
# network.to(device)
# network.load_state_dict(torch.load('model_e5_202003111707.pt'))
# network.load_state_dict(torch.load('model_e1_202003112054.pt'))
# network.load_state_dict(torch.load('model_e2_202003112215.pt'))
# network.load_state_dict(torch.load('model_e1_202003112328.pt'))

<All keys matched successfully>

#### Count parameters

In [ ]:
# def count_parameters(model):
#     total_param = 0
#     for name, param in model.named_parameters():
#         if param.requires_grad:
#             num_param = np.prod(param.size())
#             if param.dim() > 1:
#                 print(name, ':', 'x'.join(str(x) for x in list(param.size())), '=', num_param)
#             else:
#                 print(name, ':', num_param)
#             total_param += num_param
#     return total_param

# count_parameters(network)

### Network training

In [ ]:
network.train()
network.to(device)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 03:46


current loss: 3.8027: 100%|██████████| 6424/6424 [20:00<00:00,  5.35it/s]


Train Loss: 3.7048	Train Accuracy: 0.2932	Val Loss: 4.0157	Val Accuracy: 0.2500
Train:  2020-03-11 04:10


current loss: 2.6836: 100%|██████████| 6424/6424 [20:23<00:00,  5.25it/s]


Train Loss: 2.4319	Train Accuracy: 0.4954	Val Loss: 2.9258	Val Accuracy: 0.4196
Train:  2020-03-11 04:34


...

Train Loss: 1.8591	Train Accuracy: 0.5965	Val Loss: 2.5647	Val Accuracy: 0.4676
Train:  2020-03-11 04:58


current loss: 1.8336: 100%|██████████| 6424/6424 [29:27<00:00,  3.63it/s]


Train Loss: 1.3872	Train Accuracy: 0.6887	Val Loss: 2.3018	Val Accuracy: 0.5274
Train:  2020-03-11 05:35


current loss: 1.5706: 100%|██████████| 6424/6424 [21:06<00:00,  5.07it/s]


Train Loss: 1.1834	Train Accuracy: 0.7248	Val Loss: 2.2482	Val Accuracy: 0.5443


In [ ]:
numEpochs = 3
learningRate = 5e-2
weightDecay = 5e-5
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 06:03


current loss: 1.3798: 100%|██████████| 6424/6424 [19:57<00:00,  5.36it/s]


Train Loss: 1.0247	Train Accuracy: 0.7541	Val Loss: 2.3139	Val Accuracy: 0.5367
Train:  2020-03-11 06:26


current loss: 1.2325: 100%|██████████| 6424/6424 [20:01<00:00,  5.35it/s]


Train Loss: 0.8020	Train Accuracy: 0.8025	Val Loss: 2.2546	Val Accuracy: 0.5539
Train:  2020-03-11 06:50


current loss: 1.0981: 100%|██████████| 6424/6424 [20:00<00:00,  5.35it/s]


Train Loss: 0.6960	Train Accuracy: 0.8238	Val Loss: 2.3177	Val Accuracy: 0.5567


In [ ]:
numEpochs = 5
learningRate = 2e-2
weightDecay = 1e-4
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 07:14


current loss: 0.3733: 100%|██████████| 6424/6424 [20:09<00:00,  5.31it/s]


Train Loss: 0.1972	Train Accuracy: 0.9556	Val Loss: 1.9207	Val Accuracy: 0.6289
Train:  2020-03-11 07:38


current loss: 0.2930: 100%|██████████| 6424/6424 [20:16<00:00,  5.28it/s]


Train Loss: 0.1631	Train Accuracy: 0.9637	Val Loss: 1.9866	Val Accuracy: 0.6276
Train:  2020-03-11 08:01


...

Train Loss: 0.1849	Train Accuracy: 0.9546	Val Loss: 2.0568	Val Accuracy: 0.6187
Train:  2020-03-11 08:25


current loss: 0.3962: 100%|██████████| 6424/6424 [20:16<00:00,  5.28it/s]


Train Loss: 0.2685	Train Accuracy: 0.9288	Val Loss: 2.2219	Val Accuracy: 0.5902
Train:  2020-03-11 08:49


current loss: 0.5061: 100%|██████████| 6424/6424 [20:15<00:00,  5.28it/s]


Train Loss: 0.3215	Train Accuracy: 0.9147	Val Loss: 2.2178	Val Accuracy: 0.5826


In [ ]:
numEpochs = 5
learningRate = 1e-2
weightDecay = 1e-4
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 09:13


current loss: 0.1160: 100%|██████████| 6424/6424 [20:08<00:00,  5.32it/s]


Train Loss: 0.0590	Train Accuracy: 0.9899	Val Loss: 1.8709	Val Accuracy: 0.6498
Train:  2020-03-11 09:37


current loss: 0.0665: 100%|██████████| 6424/6424 [20:05<00:00,  5.33it/s]


Train Loss: 0.0428	Train Accuracy: 0.9940	Val Loss: 1.8711	Val Accuracy: 0.6578
Train:  2020-03-11 10:01


...

Train Loss: 0.0276	Train Accuracy: 0.9974	Val Loss: 1.8128	Val Accuracy: 0.6633
Train:  2020-03-11 10:25


current loss: 0.0388: 100%|██████████| 6424/6424 [20:06<00:00,  5.32it/s]


Train Loss: 0.0229	Train Accuracy: 0.9985	Val Loss: 1.7436	Val Accuracy: 0.6717
Train:  2020-03-11 10:48


current loss: 0.0328: 100%|██████████| 6424/6424 [20:07<00:00,  5.32it/s]


Train Loss: 0.0201	Train Accuracy: 0.9992	Val Loss: 1.6986	Val Accuracy: 0.6776


In [ ]:
numEpochs = 5
learningRate = 5e-3
weightDecay = 2e-4
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 11:12


current loss: 0.0238: 100%|██████████| 6424/6424 [20:10<00:00,  5.31it/s]


Train Loss: 0.0172	Train Accuracy: 0.9997	Val Loss: 1.6386	Val Accuracy: 0.6789
Train:  2020-03-11 11:36


current loss: 0.0277: 100%|██████████| 6424/6424 [20:12<00:00,  5.30it/s]


Train Loss: 0.0187	Train Accuracy: 0.9998	Val Loss: 1.5750	Val Accuracy: 0.6911
Train:  2020-03-11 12:00


...

Train Loss: 0.0225	Train Accuracy: 0.9999	Val Loss: 1.5507	Val Accuracy: 0.6896
Train:  2020-03-11 12:24


current loss: 0.0384: 100%|██████████| 6424/6424 [20:15<00:00,  5.29it/s]


Train Loss: 0.0257	Train Accuracy: 0.9999	Val Loss: 1.5390	Val Accuracy: 0.6896
Train:  2020-03-11 12:48


current loss: 0.0444: 100%|██████████| 6424/6424 [19:59<00:00,  5.36it/s]


Train Loss: 0.0295	Train Accuracy: 0.9999	Val Loss: 1.5353	Val Accuracy: 0.6885


#### CSV checkpoint 1

In [ ]:
numEpochs = 10
learningRate = 2e-3
weightDecay = 2e-4
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 15:07


current loss: 0.0316: 100%|██████████| 6424/6424 [20:20<00:00,  5.26it/s]


Train Loss: 0.0239	Train Accuracy: 1.0000	Val Loss: 1.5069	Val Accuracy: 0.6965
Train:  2020-03-11 15:31


current loss: 0.0315: 100%|██████████| 6424/6424 [20:14<00:00,  5.29it/s]


Train Loss: 0.0230	Train Accuracy: 1.0000	Val Loss: 1.4969	Val Accuracy: 0.6954
Train:  2020-03-11 15:55


...

current loss: 0.0333: 100%|██████████| 6424/6424 [20:14<00:00,  5.29it/s]


Train Loss: 0.0252	Train Accuracy: 1.0000	Val Loss: 1.4877	Val Accuracy: 0.7002
Train:  2020-03-11 16:43


current loss: 0.0339: 100%|██████████| 6424/6424 [20:24<00:00,  5.25it/s]


Train Loss: 0.0251	Train Accuracy: 1.0000	Val Loss: 1.4843	Val Accuracy: 0.6978
Train:  2020-03-11 17:07


current loss: 0:   5%|▍         | 307/6424 [00:58<19:24,  5.25it/s]

#### CSV checkpoint 2

In [ ]:
numEpochs = 2
learningRate = 5e-2
weightDecay = 1e-4
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 18:02


current loss: 1.9112: 100%|██████████| 6424/6424 [20:53<00:00,  5.13it/s]


Train Loss: 1.6200	Train Accuracy: 0.6332	Val Loss: 2.5037	Val Accuracy: 0.4848
Train:  2020-03-11 18:27


current loss: 1.5758: 100%|██████████| 6424/6424 [20:55<00:00,  5.12it/s]


Train Loss: 1.1111	Train Accuracy: 0.7359	Val Loss: 2.2514	Val Accuracy: 0.5437


In [ ]:
numEpochs = 2
learningRate = 2e-2
weightDecay = 1e-4
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 18:51


current loss: 0.6535: 100%|██████████| 6424/6424 [20:41<00:00,  5.18it/s]


Train Loss: 0.3739	Train Accuracy: 0.9115	Val Loss: 1.8128	Val Accuracy: 0.6417
Train:  2020-03-11 19:15


current loss: 0.5339: 100%|██████████| 6424/6424 [20:38<00:00,  5.19it/s]


Train Loss: 0.2961	Train Accuracy: 0.9266	Val Loss: 1.9001	Val Accuracy: 0.6274


In [ ]:
numEpochs = 2
learningRate = 1e-2
weightDecay = 1e-4
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 19:40


current loss: 0.1721: 100%|██████████| 6424/6424 [20:59<00:00,  5.10it/s]


Train Loss: 0.0860	Train Accuracy: 0.9836	Val Loss: 1.7323	Val Accuracy: 0.6722
Train:  2020-03-11 20:04


current loss: 0.1049: 100%|██████████| 6424/6424 [21:00<00:00,  5.10it/s]


Train Loss: 0.0607	Train Accuracy: 0.9900	Val Loss: 1.7402	Val Accuracy: 0.6748


In [ ]:
numEpochs = 2
learningRate = 5e-3
weightDecay = 5e-5
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 21:27


current loss: 0.0278: 100%|██████████| 6424/6424 [20:26<00:00,  5.24it/s]


Train Loss: 0.0170	Train Accuracy: 0.9985	Val Loss: 1.7001	Val Accuracy: 0.6852
Train:  2020-03-11 21:51


current loss: 0.0189: 100%|██████████| 6424/6424 [20:35<00:00,  5.20it/s]


In [ ]:
numEpochs = 1
learningRate = 1e-3
weightDecay = 5e-5
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 22:34


current loss: 0.0131: 100%|██████████| 6424/6424 [48:36<00:00,  2.20it/s]     


Train Loss: 0.0099	Train Accuracy: 0.9996	Val Loss: 1.7009	Val Accuracy: 0.6900


#### CSV checkpoint 3

In [ ]:
numEpochs = 5
learningRate = 1e-3
weightDecay = 1e-5
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

Train:  2020-03-11 23:38


current loss: 0.0112: 100%|██████████| 6424/6424 [20:13<00:00,  5.29it/s]


Train Loss: 0.0091	Train Accuracy: 0.9997	Val Loss: 1.6838	Val Accuracy: 0.6933


#### CSV checkpoint 4

In [ ]:
numEpochs = 5
learningRate = 1e-3
weightDecay = 1e-5
optimizer = torch.optim.SGD(network.parameters(), lr=learningRate, weight_decay=weightDecay, momentum=0.9)
train(network, train_dataloader, dev_dataloader)

#### CSV checkpoint 5

## Test predictions

In [ ]:
idx_to_class = {}
for key in train_dataset.class_to_idx.keys():
    idx_to_class[train_dataset.class_to_idx[key]] = key

In [ ]:
class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        path = self.imgs[index][0]
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path

In [ ]:
test_dataset = ImageFolderWithPaths(root='test_classification/', transform=torchvision.transforms.ToTensor())
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=10, shuffle=False, num_workers=8)

def test_predictions(model, test_loader):
    model.eval()
    test_predictions = {}

    for batch_num, (feats, labels, filename) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)[1]
        
        _, pred_labels = torch.max(F.softmax(outputs, dim=1), 1)
        pred_labels = pred_labels.view(-1)
        for i in range(len(pred_labels)):
            test_predictions[filename[i]] = pred_labels[i].cpu().tolist()

    return test_predictions

test_predicted_raw_labels = test_predictions(network, test_dataloader)
len(test_predicted_raw_labels)

4600

In [ ]:
blist = [[batch_num, first] for batch_num, first in enumerate(test_dataset)]

In [ ]:
test_predicted_labels = {}
for key in test_predicted_raw_labels.keys():
    test_predicted_labels[key.replace('test_classification/medium/', '')] = idx_to_class[test_predicted_raw_labels[key]]

In [ ]:
with open("classification_" + datetime.now(pytz.timezone('US/Eastern')).strftime("%Y%m%d%H%M") + ".csv", 'w') as f:
    f.write('Id,Category\n')
    for key in test_predicted_labels.keys():
        f.write("%s,%s\n"%(key,test_predicted_labels[key]))

## Verification

In [ ]:
test_verification_file = open("test_trials_verification_student.txt")

with test_verification_file as file:
    test_verification_file_lines = file.readlines()

test_verification_first_images = []
test_verification_second_images = []
for line in test_verification_file_lines:
    line_split = line.replace('\n', '').split(' ')
    test_verification_first_images.append(line_split[0])
    test_verification_second_images.append(line_split[1])

In [ ]:
# !mkdir verification_images
# !mv test_verification verification_images

mkdir: cannot create directory ‘verification_images’: File exists
mv: cannot stat 'test_verification': No such file or directory


In [ ]:
verification_dataset = ImageFolderWithPaths(root='verification_images/', transform=torchvision.transforms.ToTensor())
verification_dataloader = torch.utils.data.DataLoader(verification_dataset, batch_size=10, shuffle=False, num_workers=8)

def test_verification_probabilities(model, test_loader):
    model.eval()
    test_features = {}
    
    for batch_num, (feats, labels, filename) in enumerate(test_loader):
        feats, labels = feats.to(device), labels.to(device)
        outputs = model(feats)[0]
        
        for i in range(len(outputs)):
            test_features[filename[i].replace('verification_images/test_verification/', '')] = outputs[i].cpu().tolist()

    return test_features

test_predicted_raw_probs = test_verification_probabilities(network, verification_dataloader)
len(test_predicted_raw_probs)

169392

In [ ]:
files_list = []
similarity_list = []

for i in range(len(test_verification_first_images)):
    f = test_verification_first_images[i]
    s = test_verification_second_images[i]
    files_list.append(f + " " + s)
    similarity_list.append(
        np.dot(test_predicted_raw_probs[f], test_predicted_raw_probs[s]) / (np.linalg.norm(test_predicted_raw_probs[f]) * np.linalg.norm(test_predicted_raw_probs[s]))
    )
files_list[0]

'262615.jpg 207587.jpg'

In [ ]:
with open("verification_" + datetime.now(pytz.timezone('US/Eastern')).strftime("%Y%m%d%H%M") + ".csv", 'w') as f:
    f.write('trial,score\n')
    for i in range(len(files_list)):
        f.write("%s,%s\n"%(files_list[i], similarity_list[i]))